In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Jack's Car Rental Problem


Jack manages two locations for a nationwide car rental company. Each day, some number of customers arrive at each location to rent cars. If Jack has a car available, he rents it out and is credited \$10 by the national company. If he is out of cars at that location, then the business is lost. Cars become available for renting the day after they are returned. To help ensure that cars are available where they are needed, Jack can move them between the two locations overnight, at a cost of \$2 per car moved. We assume that the number of cars requested and returned at each location are Poisson random variables, meaning that the probability that the number is $n$ is $\frac{\lambda^n}{n!}e^{-\lambda}$, where $\lambda $ is the expected number. Suppose $\lambda $ is 3 and 4 for rental requests at the first and second locations and 3 and 2 for returns. To simplify the problem slightly, we assume that there can be no more than 20 cars at each location (any additional cars are returned to the nationwide company, and thus disappear from the problem) and a maximum of five cars can be moved from one location to the other in one night. We take the discount rate to be and formulate this as a continuing finite MDP, where the time steps are days, the state is the number of cars at each location at the end of the day, and the actions are the net numbers of cars moved between the two locations overnight. 

In [13]:
def prob(n_rental_i, n_return_i, n_rental_j, n_return_j):    
    prb = np.double(np.exp(-12.0) * pow(3.0,n_rental_i) * pow(3.0,n_return_i) * pow(4.0,n_rental_j)\
    * pow(2.0,n_return_j)) / np.double(np.math.factorial(n_rental_i)) / np.double(np.math.factorial(n_return_i))\
    / np.double(np.math.factorial(n_rental_j)) / np.double(np.math.factorial(n_return_j))
    return prb


N = 21 # most number of cars in each place (plus 1)
V = np.zeros([N,N]) 

P = np.zeros([N,N]) # policy matrix, the value of which is how many cars to move from 2nd place to 1st place (num_car_1, num_car_2)
R = np.zeros([N,N]) # reward of one step(day), including cost

Pi = np.zeros([N,N,10]) # policy matrix in process
Vi = np.zeros([N,N,10])

gamma = 0.9 # discount factor

theta = 1.0 # error threshold


counter_whole = 0
policy_stable = 0

for k in range(5):
    
    #---------------------Policy Evaluation---------------------    
    delta = theta + 1.0 # intialization
    counter = 0
    while delta > theta:
        delta = 0
        for i in range(N):
            for j in range(N):
                tmp = 0
                total_proba_tmp = 0
                v = V[i,j]
                for ip in range(N+10): # there is a possibility of many cars has been returned, thus the cars next day>20, though will be move to national wide company, but still need to consider in computation
                    for jp in range(N+10):
                        for n_rental_i in range(i+1):
                            for n_rental_j in range(j+1):
                                n_move = P[i,j]
                                n_return_i = ip - i - n_move + n_rental_i
                                n_return_j = jp - j + n_move + n_rental_j
                                
                                if ip > N-1:
                                    ip = N-1 # the rebundant cars going to the national wide company
                                if jp > N-1:
                                    jp = N-1 # the rebundant cars going to the national wide company
                                
                                if n_return_i >= 0 and n_return_j >=0 :
                                    
                                    print(n_rental_i, int(n_return_i), n_rental_j, int(n_return_j))
                                    
                                    pb = prob(n_rental_i, n_return_i, n_rental_j, n_return_j)
                                    tmp += pb * (10.0*(n_rental_i + n_rental_j) - np.abs(2.0 * n_move))  # expected reward
                                    tmp += pb * gamma * V[ip,jp] # next step value 
                                    total_proba_tmp += pb
                                    
                delta = max(delta,abs(v-tmp))
                V[i,j] = tmp
                print('i:%d, j:%d,total_prob:%f ,V:%f'%(i,j,total_proba_tmp ,tmp),'\r',end="")
        counter += 1
        print('\n policy evaluation iteration:%d'%counter,'\n',end="")

    
    #---------------------Policy Improvement---------------------
    while not policy_stable:
        policy_stable = 1
        for i in range(N):
            for j in range(N):
                b = P[i,j]
                n_move_range = np.linspace(-5,5,11)
                tmp_move_max = 0
                n_move_best = 0
                for n_move in n_move_range:
                    n_move = int(n_move)
                    tmp_move = 0
                    for ip in range(N+10):
                        for jp in range(N+10):
                            for n_rental_i in range(i+1):
                                for n_rental_j in range(j+1):
                                
                                    n_return_i = ip - i - n_move + n_rental_i
                                    n_return_j = jp - j + n_move + n_rental_j
                                    
                                    if ip > N-1:
                                        ip = N-1 # the rebundant cars going to the national wide company
                                    if jp > N-1:
                                        jp = N-1 # the rebundant cars going to the national wide company
                                    
                                    if n_return_i >= 0 and n_return_j >=0 :
                                        pb = prob(n_rental_i, n_return_i, n_rental_j, n_return_j)
                                        tmp_move += pb * (10.0*(n_rental_i + n_rental_j) - np.abs(2.0 * n_move))  # expected reward
                                        tmp_move += pb * gamma * V[ip,jp] # next step value
                    if tmp_move > tmp_move_max:
                        n_move_best = n_move                    
                    tmp_move_max = max(tmp_move_max,tmp_move)

                        
                P[i,j] = n_move_best
                
                if not b==P[i,j]:
                    policy_stable = 0
    Vi[:,:,k] = V
    Pi[:,:,k] = P
    counter_whole += 1
    
    print('\n-------Iteration:%d'%counter_whole,'----------\n',end="")

0 0 0 0
0 0 0 1
0 0 0 2
0 0 0 3
0 0 0 4
0 0 0 5
0 0 0 6
0 0 0 7
0 0 0 8
0 0 0 9
0 0 0 10
0 0 0 11
0 0 0 12
0 0 0 13
0 0 0 14
0 0 0 15
0 0 0 16
0 0 0 17
0 0 0 18
0 0 0 19
0 0 0 20
0 0 0 21
0 0 0 22
0 0 0 23
0 0 0 24
0 0 0 25
0 0 0 26
0 0 0 27
0 0 0 28
0 0 0 29
0 0 0 30
0 1 0 0
0 1 0 1
0 1 0 2
0 1 0 3
0 1 0 4
0 1 0 5
0 1 0 6
0 1 0 7
0 1 0 8
0 1 0 9
0 1 0 10
0 1 0 11
0 1 0 12
0 1 0 13
0 1 0 14
0 1 0 15
0 1 0 16
0 1 0 17
0 1 0 18
0 1 0 19
0 1 0 20
0 1 0 21
0 1 0 22
0 1 0 23
0 1 0 24
0 1 0 25
0 1 0 26
0 1 0 27
0 1 0 28
0 1 0 29
0 1 0 30
0 2 0 0
0 2 0 1
0 2 0 2
0 2 0 3
0 2 0 4
0 2 0 5
0 2 0 6
0 2 0 7
0 2 0 8
0 2 0 9
0 2 0 10
0 2 0 11
0 2 0 12
0 2 0 13
0 2 0 14
0 2 0 15
0 2 0 16
0 2 0 17
0 2 0 18
0 2 0 19
0 2 0 20
0 2 0 21
0 2 0 22
0 2 0 23
0 2 0 24
0 2 0 25
0 2 0 26
0 2 0 27
0 2 0 28
0 2 0 29
0 2 0 30
0 3 0 0
0 3 0 1
0 3 0 2
0 3 0 3
0 3 0 4
0 3 0 5
0 3 0 6
0 3 0 7
0 3 0 8
0 3 0 9
0 3 0 10
0 3 0 11
0 3 0 12
0 3 0 13
0 3 0 14
0 3 0 15
0 3 0 16
0 3 0 17
0 3 0 18
0 3 0 19
0 3 0 20
0 3 0 21
0 3 0

0 2 1 8
0 2 0 8
0 2 1 9
0 2 0 9
0 2 1 10
0 2 0 10
0 2 1 11
0 2 0 11
0 2 1 12
0 2 0 12
0 2 1 13
0 2 0 13
0 2 1 14
0 2 0 14
0 2 1 15
0 2 0 15
0 2 1 16
0 2 0 16
0 2 1 17
0 2 0 17
0 2 1 18
0 2 0 18
0 2 1 19
0 2 0 19
0 2 1 20
0 2 0 20
0 2 1 20
0 2 0 21
0 2 1 20
0 2 0 22
0 2 1 20
0 2 0 23
0 2 1 20
0 2 0 24
0 2 1 20
0 2 0 25
0 2 1 20
0 2 0 26
0 2 1 20
0 2 0 27
0 2 1 20
0 2 0 28
0 2 1 20
0 2 0 29
0 2 1 20
0 3 1 0
0 3 0 0
0 3 1 1
0 3 0 1
0 3 1 2
0 3 0 2
0 3 1 3
0 3 0 3
0 3 1 4
0 3 0 4
0 3 1 5
0 3 0 5
0 3 1 6
0 3 0 6
0 3 1 7
0 3 0 7
0 3 1 8
0 3 0 8
0 3 1 9
0 3 0 9
0 3 1 10
0 3 0 10
0 3 1 11
0 3 0 11
0 3 1 12
0 3 0 12
0 3 1 13
0 3 0 13
0 3 1 14
0 3 0 14
0 3 1 15
0 3 0 15
0 3 1 16
0 3 0 16
0 3 1 17
0 3 0 17
0 3 1 18
0 3 0 18
0 3 1 19
0 3 0 19
0 3 1 20
0 3 0 20
0 3 1 20
0 3 0 21
0 3 1 20
0 3 0 22
0 3 1 20
0 3 0 23
0 3 1 20
0 3 0 24
0 3 1 20
0 3 0 25
0 3 1 20
0 3 0 26
0 3 1 20
0 3 0 27
0 3 1 20
0 3 0 28
0 3 1 20
0 3 0 29
0 3 1 20
0 4 1 0
0 4 0 0
0 4 1 1
0 4 0 1
0 4 1 2
0 4 0 2
0 4 1 3
0 4 0 3
0 4 1 

0 20 0 23
0 20 1 20
0 20 0 24
0 20 1 20
0 20 0 25
0 20 1 20
0 20 0 26
0 20 1 20
0 20 0 27
0 20 1 20
0 20 0 28
0 20 1 20
0 20 0 29
0 20 1 20
0 20 1 0
0 20 0 0
0 20 1 1
0 20 0 1
0 20 1 2
0 20 0 2
0 20 1 3
0 20 0 3
0 20 1 4
0 20 0 4
0 20 1 5
0 20 0 5
0 20 1 6
0 20 0 6
0 20 1 7
0 20 0 7
0 20 1 8
0 20 0 8
0 20 1 9
0 20 0 9
0 20 1 10
0 20 0 10
0 20 1 11
0 20 0 11
0 20 1 12
0 20 0 12
0 20 1 13
0 20 0 13
0 20 1 14
0 20 0 14
0 20 1 15
0 20 0 15
0 20 1 16
0 20 0 16
0 20 1 17
0 20 0 17
0 20 1 18
0 20 0 18
0 20 1 19
0 20 0 19
0 20 1 20
0 20 0 20
0 20 1 20
0 20 0 21
0 20 1 20
0 20 0 22
0 20 1 20
0 20 0 23
0 20 1 20
0 20 0 24
0 20 1 20
0 20 0 25
0 20 1 20
0 20 0 26
0 20 1 20
0 20 0 27
0 20 1 20
0 20 0 28
0 20 1 20
0 20 0 29
0 20 1 20
0 20 1 0
0 20 0 0
0 20 1 1
0 20 0 1
0 20 1 2
0 20 0 2
0 20 1 3
0 20 0 3
0 20 1 4
0 20 0 4
0 20 1 5
0 20 0 5
0 20 1 6
0 20 0 6
0 20 1 7
0 20 0 7
0 20 1 8
0 20 0 8
0 20 1 9
0 20 0 9
0 20 1 10
0 20 0 10
0 20 1 11
0 20 0 11
0 20 1 12
0 20 0 12
0 20 1 13
0 20 0 13
0 20 1 14


0 8 2 9
0 8 0 8
0 8 1 9
0 8 2 10
0 8 0 9
0 8 1 10
0 8 2 11
0 8 0 10
0 8 1 11
0 8 2 12
0 8 0 11
0 8 1 12
0 8 2 13
0 8 0 12
0 8 1 13
0 8 2 14
0 8 0 13
0 8 1 14
0 8 2 15
0 8 0 14
0 8 1 15
0 8 2 16
0 8 0 15
0 8 1 16
0 8 2 17
0 8 0 16
0 8 1 17
0 8 2 18
0 8 0 17
0 8 1 18
0 8 2 19
0 8 0 18
0 8 1 19
0 8 2 20
0 8 0 19
0 8 1 19
0 8 2 20
0 8 0 20
0 8 1 19
0 8 2 20
0 8 0 21
0 8 1 19
0 8 2 20
0 8 0 22
0 8 1 19
0 8 2 20
0 8 0 23
0 8 1 19
0 8 2 20
0 8 0 24
0 8 1 19
0 8 2 20
0 8 0 25
0 8 1 19
0 8 2 20
0 8 0 26
0 8 1 19
0 8 2 20
0 8 0 27
0 8 1 19
0 8 2 20
0 8 0 28
0 8 1 19
0 8 2 20
0 9 2 0
0 9 1 0
0 9 2 1
0 9 0 0
0 9 1 1
0 9 2 2
0 9 0 1
0 9 1 2
0 9 2 3
0 9 0 2
0 9 1 3
0 9 2 4
0 9 0 3
0 9 1 4
0 9 2 5
0 9 0 4
0 9 1 5
0 9 2 6
0 9 0 5
0 9 1 6
0 9 2 7
0 9 0 6
0 9 1 7
0 9 2 8
0 9 0 7
0 9 1 8
0 9 2 9
0 9 0 8
0 9 1 9
0 9 2 10
0 9 0 9
0 9 1 10
0 9 2 11
0 9 0 10
0 9 1 11
0 9 2 12
0 9 0 11
0 9 1 12
0 9 2 13
0 9 0 12
0 9 1 13
0 9 2 14
0 9 0 13
0 9 1 14
0 9 2 15
0 9 0 14
0 9 1 15
0 9 2 16
0 9 0 15
0 9 1 16
0 9 2 17

0 20 1 4
0 20 2 5
0 20 0 4
0 20 1 5
0 20 2 6
0 20 0 5
0 20 1 6
0 20 2 7
0 20 0 6
0 20 1 7
0 20 2 8
0 20 0 7
0 20 1 8
0 20 2 9
0 20 0 8
0 20 1 9
0 20 2 10
0 20 0 9
0 20 1 10
0 20 2 11
0 20 0 10
0 20 1 11
0 20 2 12
0 20 0 11
0 20 1 12
0 20 2 13
0 20 0 12
0 20 1 13
0 20 2 14
0 20 0 13
0 20 1 14
0 20 2 15
0 20 0 14
0 20 1 15
0 20 2 16
0 20 0 15
0 20 1 16
0 20 2 17
0 20 0 16
0 20 1 17
0 20 2 18
0 20 0 17
0 20 1 18
0 20 2 19
0 20 0 18
0 20 1 19
0 20 2 20
0 20 0 19
0 20 1 19
0 20 2 20
0 20 0 20
0 20 1 19
0 20 2 20
0 20 0 21
0 20 1 19
0 20 2 20
0 20 0 22
0 20 1 19
0 20 2 20
0 20 0 23
0 20 1 19
0 20 2 20
0 20 0 24
0 20 1 19
0 20 2 20
0 20 0 25
0 20 1 19
0 20 2 20
0 20 0 26
0 20 1 19
0 20 2 20
0 20 0 27
0 20 1 19
0 20 2 20
0 20 0 28
0 20 1 19
0 20 2 20
0 20 2 0
0 20 1 0
0 20 2 1
0 20 0 0
0 20 1 1
0 20 2 2
0 20 0 1
0 20 1 2
0 20 2 3
0 20 0 2
0 20 1 3
0 20 2 4
0 20 0 3
0 20 1 4
0 20 2 5
0 20 0 4
0 20 1 5
0 20 2 6
0 20 0 5
0 20 1 6
0 20 2 7
0 20 0 6
0 20 1 7
0 20 2 8
0 20 0 7
0 20 1 8
0 20 2 9
0 20

0 3 0 24
0 3 1 18
0 3 2 19
0 3 3 20
0 3 0 25
0 3 1 18
0 3 2 19
0 3 3 20
0 3 0 26
0 3 1 18
0 3 2 19
0 3 3 20
0 3 0 27
0 3 1 18
0 3 2 19
0 3 3 20
0 4 3 0
0 4 2 0
0 4 3 1
0 4 1 0
0 4 2 1
0 4 3 2
0 4 0 0
0 4 1 1
0 4 2 2
0 4 3 3
0 4 0 1
0 4 1 2
0 4 2 3
0 4 3 4
0 4 0 2
0 4 1 3
0 4 2 4
0 4 3 5
0 4 0 3
0 4 1 4
0 4 2 5
0 4 3 6
0 4 0 4
0 4 1 5
0 4 2 6
0 4 3 7
0 4 0 5
0 4 1 6
0 4 2 7
0 4 3 8
0 4 0 6
0 4 1 7
0 4 2 8
0 4 3 9
0 4 0 7
0 4 1 8
0 4 2 9
0 4 3 10
0 4 0 8
0 4 1 9
0 4 2 10
0 4 3 11
0 4 0 9
0 4 1 10
0 4 2 11
0 4 3 12
0 4 0 10
0 4 1 11
0 4 2 12
0 4 3 13
0 4 0 11
0 4 1 12
0 4 2 13
0 4 3 14
0 4 0 12
0 4 1 13
0 4 2 14
0 4 3 15
0 4 0 13
0 4 1 14
0 4 2 15
0 4 3 16
0 4 0 14
0 4 1 15
0 4 2 16
0 4 3 17
0 4 0 15
0 4 1 16
0 4 2 17
0 4 3 18
0 4 0 16
0 4 1 17
0 4 2 18
0 4 3 19
0 4 0 17
0 4 1 18
0 4 2 19
0 4 3 20
0 4 0 18
0 4 1 18
0 4 2 19
0 4 3 20
0 4 0 19
0 4 1 18
0 4 2 19
0 4 3 20
0 4 0 20
0 4 1 18
0 4 2 19
0 4 3 20
0 4 0 21
0 4 1 18
0 4 2 19
0 4 3 20
0 4 0 22
0 4 1 18
0 4 2 19
0 4 3 20
0 4 0 23
0 4 1

0 14 3 15
0 14 0 13
0 14 1 14
0 14 2 15
0 14 3 16
0 14 0 14
0 14 1 15
0 14 2 16
0 14 3 17
0 14 0 15
0 14 1 16
0 14 2 17
0 14 3 18
0 14 0 16
0 14 1 17
0 14 2 18
0 14 3 19
0 14 0 17
0 14 1 18
0 14 2 19
0 14 3 20
0 14 0 18
0 14 1 18
0 14 2 19
0 14 3 20
0 14 0 19
0 14 1 18
0 14 2 19
0 14 3 20
0 14 0 20
0 14 1 18
0 14 2 19
0 14 3 20
0 14 0 21
0 14 1 18
0 14 2 19
0 14 3 20
0 14 0 22
0 14 1 18
0 14 2 19
0 14 3 20
0 14 0 23
0 14 1 18
0 14 2 19
0 14 3 20
0 14 0 24
0 14 1 18
0 14 2 19
0 14 3 20
0 14 0 25
0 14 1 18
0 14 2 19
0 14 3 20
0 14 0 26
0 14 1 18
0 14 2 19
0 14 3 20
0 14 0 27
0 14 1 18
0 14 2 19
0 14 3 20
0 15 3 0
0 15 2 0
0 15 3 1
0 15 1 0
0 15 2 1
0 15 3 2
0 15 0 0
0 15 1 1
0 15 2 2
0 15 3 3
0 15 0 1
0 15 1 2
0 15 2 3
0 15 3 4
0 15 0 2
0 15 1 3
0 15 2 4
0 15 3 5
0 15 0 3
0 15 1 4
0 15 2 5
0 15 3 6
0 15 0 4
0 15 1 5
0 15 2 6
0 15 3 7
0 15 0 5
0 15 1 6
0 15 2 7
0 15 3 8
0 15 0 6
0 15 1 7
0 15 2 8
0 15 3 9
0 15 0 7
0 15 1 8
0 15 2 9
0 15 3 10
0 15 0 8
0 15 1 9
0 15 2 10
0 15 3 11
0 15 0 9


KeyboardInterrupt: 

In [ ]:
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='col', sharey='row')
ax1.pcolor(Pi[:,:,0])
ax2.pcolor(Pi[:,:,1])
ax3.pcolor(Pi[:,:,2])
ax4.pcolor(Pi[:,:,3])

plt.figure()
plt.pcolor(P)
plt.colorbar()

plt.show()



In [ ]:
plt.pcolor(V)
plt.colorbar()
plt.show()

In [9]:
pppp = 0
for aaa in range(40):
    for bbb in range(40):
        for ccc in range(40):
            for ddd in range(40):
                pppp += prob(aaa,bbb,ccc,ddd)
pppp

0.99999999999975686